In [1]:
from collections import defaultdict

In [2]:
with open('12-07-input','r') as f:
    lines = [x for x in f]

In [3]:
lines[:10]

['$ cd /\n',
 '$ ls\n',
 'dir bfqzjjct\n',
 'dir cgcqpjpn\n',
 'dir frrjh\n',
 'dir jjl\n',
 '293559 jztrccm.hvd\n',
 'dir mqvn\n',
 '$ cd bfqzjjct\n',
 '$ ls\n']

### Part One

In [4]:
adj_list = defaultdict(list)

vertices = {'/': {'parent': None, 'kind': 'dir', 'size': 0 }}

working_dir = ['/']
current_dir = working_dir[-1]

#  Build the file system as a tree-type of structure

for command in lines:
    parts = [x.strip() for x in command.split()]
    if command[0] == '$':
        if parts[1] == 'cd':
            if parts[2] == '..':
                working_dir.pop()
                current_dir = working_dir[-1]
            elif parts[2] == '/':
                working_dir = ['/']
                current_dir == '/'
            else:
                new_dir = current_dir + '/' + parts[2]
                if new_dir not in vertices:
                    print(f'Adding directory {new_dir}')
                    vertices[new_dir] = {'parent': current_dir, 'kind': 'dir', 'size': 0}
                working_dir.append(new_dir)
                current_dir = new_dir
        elif parts[1] == 'ls':
            continue
        else:
            raise ValueError(f'Unknown command encountered {command}')
    elif command[0] == 'd':  # Encountered a subdirectory
        new_dir = current_dir + '/' + parts[1]
        if new_dir not in vertices:
            vertices[new_dir] = {'parent': current_dir, 'kind': 'dir', 'size':0}
            adj_list[current_dir].append(new_dir)
    elif command[0] in '0123456789':
        new_file = current_dir + '/' + parts[1]
        if new_file not in vertices:
            vertices[new_file] = {'parent': current_dir, 'kind': 'file', 'size': int(parts[0])}
            adj_list[current_dir].append(new_file)
    else:
        raise ValueError(f'Unknown command encountered {command}')  

###  Depth-first search

In [5]:
visited = defaultdict(lambda: False)
visited['/'] = True

stack = ['/']

while stack:
    top = stack[-1]
    for v in adj_list[top]:
        if (not visited[v]) and (vertices[v]['kind'] == 'file'):
            vertices[vertices[v]['parent']]['size'] += vertices[v]['size']
            visited[v] = True
        elif (not visited[v]):
            visited[v] = True
            stack.append(v)
            break
    if top == stack[-1]:
        if vertices[top]['parent']:
            vertices[vertices[top]['parent']]['size'] += vertices[top]['size']
        stack.pop()

In [6]:
sum([vertices[v]['size'] for v in vertices if vertices[v]['kind'] == 'dir' and vertices[v]['size'] <= 100000])

1453349

### Part Two

In [7]:
vertices['/']

{'parent': None, 'kind': 'dir', 'size': 42586708}

In [8]:
total_space = 70000000

In [9]:
unused_space = total_space - vertices['/']['size']

In [10]:
needed_space = 30000000 - unused_space

In [11]:
min([vertices[v]['size'] for v in vertices if vertices[v]['size'] >= needed_space])

2948823